#### 数据准备
1.将数据从dicom文件保存为npy文件

#### 导入模块

In [1]:
import os
import glob
import cv2

from tqdm import tqdm
import SimpleITK as sitk

import cv2
import numpy as np
import pandas as pd

#### config

In [5]:
class_info_path = r'F:/RSNA/data/stage_1_detailed_class_info.csv'
train_labels_path = r'F:/RSNA/data/stage_1_train_labels.csv'

In [4]:
class_info = pd.read_csv(class_info_path)

In [6]:
train_labels =  pd.read_csv(train_labels_path)

In [7]:
class_info.head()

,patientId,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,No Lung Opacity / Not Normal
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,No Lung Opacity / Not Normal
2,00322d4d-1c29-4943-afc9-b6754be640eb,No Lung Opacity / Not Normal
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,Normal
4,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity


In [8]:
train_labels[:50]

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
5,00436515-870c-4b36-a041-de91049b9ab4,562.0,152.0,256.0,453.0,1
6,00569f44-917d-4c86-a842-81832af98c30,NaN,NaN,NaN,NaN,0
7,006cec2e-6ce2-4549-bffa-eadfcd1e9970,NaN,NaN,NaN,NaN,0
8,00704310-78a8-4b38-8475-49f4573b2dbb,323.0,577.0,160.0,104.0,1
9,00704310-78a8-4b38-8475-49f4573b2dbb,695.0,575.0,162.0,137.0,1


#### 分组
* 测试集未知
* 训练集：
    * 肺炎 具有bbox
    * 正常 
    * 其他疾病 非肺炎

In [21]:
labels = train_labels[train_labels['patientId'] == '00436515-870c-4b36-a041-de91049b9ab4']
labels

,patientId,x,y,width,height,Target
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
5,00436515-870c-4b36-a041-de91049b9ab4,562.0,152.0,256.0,453.0,1


In [22]:
labels.iloc[0]

patientId    00436515-870c-4b36-a041-de91049b9ab4
x                                             264
y                                             152
width                                         213
height                                        379
Target                                          1
Name: 4, dtype: object

#### 加载dicom数据

In [23]:
data_root = r'F:\RSNA\data'

In [24]:
data_fpaths = glob.glob(data_root+r'\**\*.dcm',recursive=True)
data_fpaths[:100]

['F:\\RSNA\\data\\stage_1_test_images\\000924cf-0f8d-42bd-9158-1af53881a557.dcm',
 'F:\\RSNA\\data\\stage_1_test_images\\000db696-cf54-4385-b10b-6b16fbb3f985.dcm',
 'F:\\RSNA\\data\\stage_1_test_images\\000fe35a-2649-43d4-b027-e67796d412e0.dcm',
 'F:\\RSNA\\data\\stage_1_test_images\\001031d9-f904-4a23-b3e5-2c088acd19c6.dcm',
 'F:\\RSNA\\data\\stage_1_test_images\\0010f549-b242-4e94-87a8-57d79de215fc.dcm',
 'F:\\RSNA\\data\\stage_1_test_images\\001916b8-3d30-4935-a5d1-8eaddb1646cd.dcm',
 'F:\\RSNA\\data\\stage_1_test_images\\0022073f-cec8-42ec-ab5f-bc2314649235.dcm',
 'F:\\RSNA\\data\\stage_1_test_images\\0022995a-45eb-4cfa-9a59-cd15f5196c64.dcm',
 'F:\\RSNA\\data\\stage_1_test_images\\0025d2de-bd78-4d36-9f72-e15a5e22ca82.dcm',
 'F:\\RSNA\\data\\stage_1_test_images\\00293de0-a530-41dc-9621-0b3def01d06d.dcm',
 'F:\\RSNA\\data\\stage_1_test_images\\002c591d-df62-4e34-8eda-838c664430a9.dcm',
 'F:\\RSNA\\data\\stage_1_test_images\\002cb550-2e31-42f1-a29d-fbc279977e71.dcm',
 'F:\\RSNA\\data

In [ ]:
for path in  tqdm(data_fpaths):
    Image = sitk.ReadImage(path)
    img_array = sitk.GetArrayFromImage(Image)[0]
    id = path.split('\\')[-1][:-4]
    #直接保存测试集
    if 'test' in path:
        save_path = os.path.join(data_root,'test',f'{id}.png')
        cv2.imwrite(save_path,img_array)
    else:
        labels = train_labels[train_labels['patientId'] == id]
        classes = class_info[class_info['patientId'] == id]
        num_label = len(labels.dropna())
        if num_label == 0:
            class_label = classes.iloc[0]['class']
            if class_label == 'Normal':
                save_path = os.path.join(data_root,'negative','normal',f'{id}.png')
                cv2.imwrite(save_path,img_array)
            else:
                save_path = os.path.join(data_root,'negative','notnormal',f'{id}.png')
                cv2.imwrite(save_path,img_array)
        else:
            bboxs = []
            copy_img = img_array.copy()
            for n in range(num_label):
                label = labels.iloc[n]
                x = label['x'] #upper-left
                y = label['y'] #upper-left y
                width = label['width'] 
                height = label['height']
                #bbox coord : (upper-left y,upper-left x,bottom-right y,bottom-right x)
                bbox = [y,x,y+height,x+width]
                bboxs.append(bbox)
#                 cv2.rectangle(copy_img,(int(x),int(y)),(int(x+width),int(y+height)),
#                               [0,255,0],2)
#                 cv2.imshow('DR',copy_img)
#                 cv2.waitKey(0)
            bboxs = np.array(bboxs)
            #验证集稍后分裂
            save_path = os.path.join(data_root,'train',f'{id}.npy')
            np.save(save_path,{'image':img_array,'bboxs':bboxs})

#### NEXT

* 使用faster RCNN 对肺炎训练集 训练得到基准模型


#### 视频到此结束